In [1]:
import pandas as pd
import numpy as np


import pyspark
from pydataset import data

from pyspark.sql.functions import col, expr, concat, lit, regexp_extract, regexp_replace, slice, sum, avg, min, max, count, mean


In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 21:30:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
# create pandas dataframe

pd_df = pd.DataFrame({'language': ['Python', 'Java', 'PHP', 'C++']})

# peak into a dataframe
pd_df

,language
0,Python
1,Java
2,PHP
3,C++


In [4]:
# create spark dataframe
df = spark.createDataFrame(pd_df)

# view the schema of a dataframe
df.schema

StructType([StructField('language', StringType(), True)])

In [5]:
# get shape
df.count(), len(df.columns)

(4, 1)

In [6]:
# show the 5 first records of dataframe
df.show(5)

+--------+
|language|
+--------+
|  Python|
|    Java|
|     PHP|
|     C++|
+--------+



#### 2. Load the mpg dataset as a spark dataframe.

- a. Create 1 column of output that contains a message like the one below:
    The 1999 audi a4 has a 4 cylinder engine.

For each vehicle.

- b. Transform the trans column so that it only contains either manual or auto.

In [7]:
# load mpg dataset as a spark dataframe
mpg = spark.createDataFrame(data('mpg'))

# peak into a dataframe
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [8]:
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), 
            mpg.model, lit(' has a '), lit(' '), mpg.cyl, lit(' cyclinder engine')).alias ('message')).show(5, truncate=False)

+------------------------------------------+
|message                                   |
+------------------------------------------+
|The 1999 audi a4 has a  4 cyclinder engine|
|The 1999 audi a4 has a  4 cyclinder engine|
|The 2008 audi a4 has a  4 cyclinder engine|
|The 2008 audi a4 has a  4 cyclinder engine|
|The 1999 audi a4 has a  6 cyclinder engine|
+------------------------------------------+
only showing top 5 rows



b. Transform the trans column so that it only contains either manual or auto

In [15]:
mpg.filter(mpg.trans.like('auto%')).show(5)

+------------+----------+-----+----+---+--------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|   trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+--------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|auto(av)|  f| 21| 30|  p|compact|
|        audi|        a4|  2.8|1999|  6|auto(l5)|  f| 16| 26|  p|compact|
|        audi|        a4|  3.1|2008|  6|auto(av)|  f| 18| 27|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|auto(l5)|  4| 16| 25|  p|compact|
+------------+----------+-----+----+---+--------+---+---+---+---+-------+
only showing top 5 rows



In [16]:
mpg.filter(mpg.trans.like('manual%')).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|a4 quattro|  2.0|2008|  4|manual(m6)|  4| 20| 28|  p|compact|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### 3. Load the tips dataset as a spark dataframe.

- a. What percentage of observations are smokers?
- b. Create a column that contains the tip percentage
- c. Calculate the average tip percentage for each combination of sex and smoker.

In [17]:
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [25]:
smoker_df = tips.filter(tips.smoker=='Yes')
smoker_df.select(((count(smoker_df.smoker)/tips.count())*100 ).alias ('percentage of smoker')).show()

+--------------------+
|percentage of smoker|
+--------------------+
|  38.114754098360656|
+--------------------+



In [41]:
tips.select(col('total_bill'), col('tip'), ((tips.tip/tips.total_bill)* 100).alias ('tip percentage')).show()

+----------+----+------------------+
|total_bill| tip|    tip percentage|
+----------+----+------------------+
|     16.99|1.01|5.9446733372572105|
|     10.34|1.66|16.054158607350097|
|     21.01| 3.5|16.658733936220845|
|     23.68|3.31| 13.97804054054054|
|     24.59|3.61|14.680764538430255|
|     25.29|4.71| 18.62396204033215|
|      8.77| 2.0| 22.80501710376283|
|     26.88|3.12|11.607142857142858|
|     15.04|1.96|13.031914893617023|
|     14.78|3.23|21.853856562922868|
|     10.27|1.71| 16.65043816942551|
|     35.26| 5.0|14.180374361883155|
|     15.42|1.57|10.181582360570687|
|     18.43| 3.0|16.277807921866522|
|     14.83|3.02|20.364126770060686|
|     21.58|3.92|18.164967562557923|
|     10.33|1.67| 16.16650532429816|
|     16.29|3.71|22.774708410067525|
|     16.97| 3.5|20.624631703005306|
|     20.65|3.35|16.222760290556902|
+----------+----+------------------+
only showing top 20 rows



In [26]:
smoker_df.show()

+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|     38.01| 3.0|  Male|   Yes| Sat|Dinner|   4|
|     11.24|1.76|  Male|   Yes| Sat|Dinner|   2|
|     20.29|3.21|  Male|   Yes| Sat|Dinner|   2|
|     13.81| 2.0|  Male|   Yes| Sat|Dinner|   2|
|     11.02|1.98|  Male|   Yes| Sat|Dinner|   2|
|     18.29|3.76|  Male|   Yes| Sat|Dinner|   4|
|      3.07| 1.0|Female|   Yes| Sat|Dinner|   1|
|     15.01|2.09|  Male|   Yes| Sat|Dinner|   2|
|     26.86|3.14|Female|   Yes| Sat|Dinner|   2|
|     25.28| 5.0|Female|   Yes| Sat|Dinner|   2|
|     17.92|3.08|  Male|   Yes| Sat|Dinner|   2|
|     19.44| 3.0|  Male|   Yes|Thur| Lunch|   2|
|     32.68| 5.0|  Male|   Yes|Thur| Lunch|   2|
|     28.97| 3.0|  Male|   Yes| Fri|Dinner|   2|
|      5.75| 1.0|Female|   Yes| Fri|Dinner|   2|
|     16.32| 4.3|Female|   Yes| Fri|Dinner|   2|
|     40.17|4.73|  Male|   Yes| Fri|Dinner|   4|
|     27.28| 4.0|  M

In [43]:
male_smoker = tips.filter((tips.sex == "Male") & (tips.smoker == 'Yes'))
male_smoker.show()

+----------+----+----+------+----+------+----+
|total_bill| tip| sex|smoker| day|  time|size|
+----------+----+----+------+----+------+----+
|     38.01| 3.0|Male|   Yes| Sat|Dinner|   4|
|     11.24|1.76|Male|   Yes| Sat|Dinner|   2|
|     20.29|3.21|Male|   Yes| Sat|Dinner|   2|
|     13.81| 2.0|Male|   Yes| Sat|Dinner|   2|
|     11.02|1.98|Male|   Yes| Sat|Dinner|   2|
|     18.29|3.76|Male|   Yes| Sat|Dinner|   4|
|     15.01|2.09|Male|   Yes| Sat|Dinner|   2|
|     17.92|3.08|Male|   Yes| Sat|Dinner|   2|
|     19.44| 3.0|Male|   Yes|Thur| Lunch|   2|
|     32.68| 5.0|Male|   Yes|Thur| Lunch|   2|
|     28.97| 3.0|Male|   Yes| Fri|Dinner|   2|
|     40.17|4.73|Male|   Yes| Fri|Dinner|   4|
|     27.28| 4.0|Male|   Yes| Fri|Dinner|   2|
|     12.03| 1.5|Male|   Yes| Fri|Dinner|   2|
|     21.01| 3.0|Male|   Yes| Fri|Dinner|   2|
|     15.36|1.64|Male|   Yes| Sat|Dinner|   2|
|     20.49|4.06|Male|   Yes| Sat|Dinner|   2|
|     25.21|4.29|Male|   Yes| Sat|Dinner|   2|
|      16.0| 

In [51]:
male_smoker.select(avg((tips.tip/tips.total_bill)* 100).alias('avg tip percentage of male smoker')).show()

+---------------------------------+
|avg tip percentage of male smoker|
+---------------------------------+
|               15.277117520248513|
+---------------------------------+



In [35]:
female_smoker = tips.filter((tips.sex == "Female") & (tips.smoker == 'Yes'))
female_smoker.show()

+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|      3.07| 1.0|Female|   Yes| Sat|Dinner|   1|
|     26.86|3.14|Female|   Yes| Sat|Dinner|   2|
|     25.28| 5.0|Female|   Yes| Sat|Dinner|   2|
|      5.75| 1.0|Female|   Yes| Fri|Dinner|   2|
|     16.32| 4.3|Female|   Yes| Fri|Dinner|   2|
|     11.35| 2.5|Female|   Yes| Fri|Dinner|   2|
|     15.38| 3.0|Female|   Yes| Fri|Dinner|   2|
|      44.3| 2.5|Female|   Yes| Sat|Dinner|   3|
|     22.42|3.48|Female|   Yes| Sat|Dinner|   2|
|     14.31| 4.0|Female|   Yes| Sat|Dinner|   2|
|     17.51| 3.0|Female|   Yes| Sun|Dinner|   2|
|     10.59|1.61|Female|   Yes| Sat|Dinner|   2|
|     10.63| 2.0|Female|   Yes| Sat|Dinner|   2|
|       9.6| 4.0|Female|   Yes| Sun|Dinner|   2|
|      20.9| 3.5|Female|   Yes| Sun|Dinner|   3|
|     18.15| 3.5|Female|   Yes| Sun|Dinner|   3|
|     19.81|4.19|Female|   Yes|Thur| Lunch|   2|
|     43.11| 5.0|Fem

In [52]:
male_smoker.select(avg((tips.tip/tips.total_bill)* 100).alias('avg tip percentage of female smoker')).show()

+-----------------------------------+
|avg tip percentage of female smoker|
+-----------------------------------+
|                 15.277117520248513|
+-----------------------------------+

